In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA

# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 
import scipy.io
data = pd.Dataframe.scipy.io.loadmat('drive/MyDrive/Sem 8/DS/Exp 2/water_dataset.mat')

# train = pd.read_csv('drive/MyDrive/Sem 8/DS/Exp 2/train.csv')
# labels=train["SalePrice"]
# test = pd.read_csv('drive/MyDrive/Sem 8/DS/Exp 2/test.csv')
# data = pd.concat([train,test],ignore_index=True)
# data = data.drop("SalePrice", 1)
# ids = test["Id"]

Mounted at /content/drive


In [ ]:
type(data)

<class 'dict'>

In [ ]:
data

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data.head()

AttributeError: ignored

In [ ]:
data.shape

(2919, 80)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [ ]:
pd.set_option('max_rows', 9999)
print(data.isnull().sum().sort_values())

Id                  0
Foundation          0
HeatingQC           0
CentralAir          0
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
FullBath            0
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
TotRmsAbvGrd        0
Fireplaces          0
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
ExterCond           0
ExterQual           0
Heating             0
SaleCondition       0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
BldgType            0
RoofStyle           0
RoofMatl            0
Condition2          0
Condition1          0
OverallQual         0
Neighborhood        0
LotConfig           0
LandContour         0
LotShape            0
Street              0
LotArea             0
MSSubClass          0
LandSlope           0
HouseStyle

In [ ]:
# data = data.dropna(subset=['BsmtFinSF1'])
# data = data.dropna(subset=['GarageCars'])
# data = data.dropna(subset=['BsmtUnfSF'])
# data = data.dropna(subset=['GarageArea'])
# data = data.dropna(subset=['BsmtFinSF2'])

# data = data.dropna(subset=['TotalBsmtSF'])
# data = data.dropna(subset=['BsmtFullBath'])
# data = data.dropna(subset=['BsmtHalfBath'])
# data = data.dropna(subset=['MasVnrArea'])
# data = data.dropna(subset=['GarageYrBlt'])
# data = data.dropna(subset=['LotFrontage'])


In [ ]:
# Remove id and columns with more than a thousand missing values
data=data.drop("Id", 1)
data=data.drop("Alley", 1)
data=data.drop("Fence", 1)
data=data.drop("MiscFeature", 1)
data=data.drop("PoolQC", 1)
data=data.drop("FireplaceQu", 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a futur

In [ ]:
# Count the column types
data.dtypes.value_counts()

object     38
int64      25
float64    11
dtype: int64

In [ ]:
all_columns = data.columns.values
non_categorical = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
                   "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                   "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
                   "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                   "ScreenPorch","PoolArea", "MiscVal"]

categorical = [value for value in all_columns if value not in non_categorical]

In [ ]:
data = pd.get_dummies(data)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
data = imp.fit_transform(data)

# Log transformation
data = np.log(data)
labels = np.log(labels)

# Change -inf to 0 again
data[data==-np.inf]=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [ ]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([0.2248857 , 0.40281429, 0.52425789, 0.62418823, 0.69580422,
       0.75944463, 0.8116806 , 0.85647038, 0.89178708, 0.92273755,
       0.94898868, 0.95842727, 0.96637545, 0.97380464, 0.97971901,
       0.98501952, 0.98918839, 0.99199181, 0.99386559, 0.99520919,
       0.99611479, 0.99695667, 0.99771023, 0.99842564, 0.9989402 ,
       0.99933882, 0.99959949, 0.99978254, 0.99988174, 0.99993998,
       0.99998599, 0.99999658, 0.99999871, 0.99999943, 0.99999999,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

In [ ]:
pca = PCA(n_components=36,whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)

In [ ]:
# Split training and test
train = data[:1460]
test = data[1460:]

trainPCA = dataPCA[:1460]
testPCA = dataPCA[1460:]

In [ ]:
def test_model(clf, train):
  cv = KFold(n_splits=5,shuffle=True,random_state=45)
  r2 = make_scorer(r2_score)
  r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
  scores=[r2_val_score.mean()]
  return scores
clf = linear_model.LinearRegression()
r=test_model(clf, train)
rPCA=test_model(clf, trainPCA)

In [ ]:
r

[0.8537898788799989]

In [ ]:
rPCA

[0.8537898788799987]

In [ ]:
train.shape

(1460, 270)

In [ ]:
trainPCA.shape

(1460, 36)